In [15]:
import time
import re
import pylab as pl
import numpy as np
import pylab as pl

from datetime import datetime, timedelta

import subprocess

import os
import pandas as pd

import concurrent.futures


from joblib import Parallel, delayed
import shutil


from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import subprocess

In [16]:
os.system('gcloud compute instances delete --zone=europe-west3-c --quiet $(gcloud compute instances list --format="value(name)")')
os.system('gcloud compute instances delete --zone=asia-south1-c --quiet $(gcloud compute instances list --format="value(name)")')
os.system('gcloud compute instances delete --zone=us-west1-b --quiet $(gcloud compute instances list --format="value(name)")')   



ERROR: (gcloud.compute.instances.delete) Could not fetch resource:
 - The resource 'projects/ucr-ursa-major-lesani-lab/zones/europe-west3-c/instances/resdb-000' was not found
 - The resource 'projects/ucr-ursa-major-lesani-lab/zones/europe-west3-c/instances/resdb-001' was not found
 - The resource 'projects/ucr-ursa-major-lesani-lab/zones/europe-west3-c/instances/resdb-002' was not found
 - The resource 'projects/ucr-ursa-major-lesani-lab/zones/europe-west3-c/instances/resdb-003' was not found
 - The resource 'projects/ucr-ursa-major-lesani-lab/zones/europe-west3-c/instances/resdb-004' was not found
 - The resource 'projects/ucr-ursa-major-lesani-lab/zones/europe-west3-c/instances/resdb-005' was not found
 - The resource 'projects/ucr-ursa-major-lesani-lab/zones/europe-west3-c/instances/resdb-006' was not found
 - The resource 'projects/ucr-ursa-major-lesani-lab/zones/europe-west3-c/instances/resdb-007' was not found
 - The resource 'projects/ucr-ursa-major-lesani-lab/zones/europe-west

0

In [17]:
os.system('git add .; git commit -m "fixes"; git push -f origin HEAD:master')


[master 40f63a5] fixes
 1 file changed, 163 insertions(+), 84 deletions(-)


To github.com:draznm/resDB.git
   4da5c9e..40f63a5  HEAD -> master


0

In [18]:
config_list = ['test']


In [19]:
for cfig in config_list:    
    os.system('rm node_info.csv')
    os.system('cp node_setup/'+cfig+'.csv node_info.csv')
    
    
    df =pd.read_csv('node_info.csv')

    node_info_dict = df.to_dict()

    print(node_info_dict)

    lines  = []

    for idx, key in enumerate( node_info_dict['region'].keys()):
        cmd =  'gcloud compute instances create resdb-'+f"{idx:03}"+' --project=ucr-ursa-major-lesani-lab --zone='+node_info_dict['region'][key].strip()[1:-1]+' --machine-type=e2-standard-2 --network-interface=network-tier=PREMIUM,stack-type=IPV4_ONLY,subnet=default \
        --can-ip-forward \
        --maintenance-policy=MIGRATE \
        --provisioning-model=STANDARD \
        --service-account=961693926925-compute@developer.gserviceaccount.com \
        --scopes=https://www.googleapis.com/auth/devstorage.read_only,https://www.googleapis.com/auth/logging.write,https://www.googleapis.com/auth/monitoring.write,https://www.googleapis.com/auth/service.management.readonly,https://www.googleapis.com/auth/servicecontrol,https://www.googleapis.com/auth/trace.append \
        --tags=http-server,https-server \
        --create-disk=auto-delete=yes,boot=yes,device-name=instance-20250210-072748,image=projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20250130,mode=rw,size=20,type=pd-balanced \
        --no-shielded-secure-boot \
        --shielded-vtpm \
        --shielded-integrity-monitoring \
        --labels=goog-ec-src=vm_add-gcloud \
        --reservation-affinity=any'

        lines.append(cmd)


    lines


    def run_command(command):
        subprocess.call(command, shell=True)

    commands = lines

    # Create a ThreadPoolExecutor or ProcessPoolExecutor, depending on your needs
    with concurrent.futures.ThreadPoolExecutor(max_workers=60) as executor:
        # Submit each command to the executor for parallel execution
        futures = [executor.submit(run_command, command) for command in commands]

        # Wait for all tasks to complete
        concurrent.futures.wait(futures)
        
    time.sleep(60)
    
    os.system('gcloud compute instances list  --format="value(networkInterfaces[0].networkIP)" '+\
                  ' > temp.txt')
    
    

{'node_id': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9}, 'region': {0: " 'us-west1-b'", 1: " 'us-west1-b'", 2: " 'us-west1-b'", 3: " 'us-west1-b'", 4: " 'us-west1-b'", 5: " 'us-west1-b'", 6: " 'us-west1-b'", 7: " 'us-west1-b'", 8: " 'us-west1-b'", 9: " 'us-west1-b'"}, 'ServerOrClientregion': {0: 0, 1: 0, 2: 0, 3: 0, 4: 1, 5: 0, 6: 0, 7: 0, 8: 0, 9: 1}, 'cluster_id': {0: 0, 1: 0, 2: 0, 3: 0, 4: -1, 5: 1, 6: 1, 7: 1, 8: 1, 9: -1}, 'key_file': {0: " 'FKA.pem'", 1: " 'FKA.pem'", 2: " 'FKA.pem'", 3: " 'FKA.pem'", 4: " 'FKA.pem'", 5: "'FKA.pem'", 6: "'FKA.pem'", 7: "'FKA.pem'", 8: "'FKA.pem'", 9: " 'FKA.pem'"}, 'clientID': {0: -1, 1: -1, 2: -1, 3: -1, 4: 0, 5: -1, 6: -1, 7: -1, 8: -1, 9: 1}, 'ServerID': {0: 0, 1: 1, 2: 2, 3: 3, 4: -1, 5: 4, 6: 5, 7: 6, 8: 7, 9: -1}, ' PublicIp': {0: nan, 1: nan, 2: nan, 3: nan, 4: nan, 5: nan, 6: nan, 7: nan, 8: nan, 9: nan}}


Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/instances/resdb-007].
Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/instances/resdb-001].
 - Disk size: '20 GB' is larger than image size: '10 GB'. You might need to resize the root repartition manually if the operating system does not support automatic resizing. See https://cloud.google.com/compute/docs/disks/add-persistent-disk#resize_pd for details.



NAME       ZONE        MACHINE_TYPE   PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP   STATUS
resdb-007  us-west1-b  e2-standard-2               10.138.0.64  34.19.50.135  RUNNING


Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/instances/resdb-008].
Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/instances/resdb-003].
 - Disk size: '20 GB' is larger than image size: '10 GB'. You might need to resize the root repartition manually if the operating system does not support automatic resizing. See https://cloud.google.com/compute/docs/disks/add-persistent-disk#resize_pd for details.

Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/instances/resdb-002].


NAME       ZONE        MACHINE_TYPE   PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP  STATUS
resdb-001  us-west1-b  e2-standard-2               10.138.0.65  35.247.22.2  RUNNING
NAME       ZONE        MACHINE_TYPE   PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP    STATUS
resdb-008  us-west1-b  e2-standard-2               10.138.0.66  34.83.188.221  RUNNING


...

NAME       ZONE        MACHINE_TYPE   PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP     STATUS
resdb-009  us-west1-b  e2-standard-2               10.138.0.71  34.168.158.183  RUNNING


Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/instances/resdb-004].
Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/instances/resdb-005].
Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/instances/resdb-000].
 - Disk size: '20 GB' is larger than image size: '10 GB'. You might need to resize the root repartition manually if the operating system does not support automatic resizing. See https://cloud.google.com/compute/docs/disks/add-persistent-disk#resize_pd for details.

 - Disk size: '20 GB' is larger than image size: '10 GB'. You might need to resize the root repartition manually if the operating system does not support automatic resizing. See https://cloud.google.com/compute/docs/disks/add-persistent-disk#resize_pd for details.

 - Disk size: '20 GB' is larger than image size: '10 GB'. You might need to resize the root repartition m

NAME       ZONE        MACHINE_TYPE   PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP  STATUS
resdb-004  us-west1-b  e2-standard-2               10.138.0.73  34.127.15.0  RUNNING
NAME       ZONE        MACHINE_TYPE   PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP   STATUS
resdb-005  us-west1-b  e2-standard-2               10.138.0.69  34.82.115.35  RUNNING
NAME       ZONE        MACHINE_TYPE   PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP    STATUS
resdb-000  us-west1-b  e2-standard-2               10.138.0.75  34.168.37.201  RUNNING


Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/instances/resdb-006].
 - Disk size: '20 GB' is larger than image size: '10 GB'. You might need to resize the root repartition manually if the operating system does not support automatic resizing. See https://cloud.google.com/compute/docs/disks/add-persistent-disk#resize_pd for details.



NAME       ZONE        MACHINE_TYPE   PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP    STATUS
resdb-006  us-west1-b  e2-standard-2               10.138.0.74  34.145.27.251  RUNNING


In [20]:
with open('temp.txt') as f:
    ipList = f.read().splitlines()
ipList = [x for x in ipList if x!='None']

In [21]:
ipList

['10.138.0.75',
 '10.138.0.65',
 '10.138.0.67',
 '10.138.0.68',
 '10.138.0.73',
 '10.138.0.69',
 '10.138.0.74',
 '10.138.0.64',
 '10.138.0.66',
 '10.138.0.71']

In [22]:
    print(i)

0
1
2
3
4
5
6
7
8
9


In [24]:

new_server_only_ips= []
new_client_only_ips= []



for i in node_info_dict['node_id']:
    if node_info_dict['ServerOrClientregion'][i] ==0:
        new_server_only_ips = new_server_only_ips + [ipList[i]]
    else:
        new_client_only_ips = new_client_only_ips + [ipList[i]]



new_server_only_ips, len(new_server_only_ips), new_client_only_ips, len(new_client_only_ips) 

(['10.138.0.75',
  '10.138.0.65',
  '10.138.0.67',
  '10.138.0.68',
  '10.138.0.69',
  '10.138.0.74',
  '10.138.0.64',
  '10.138.0.66'],
 8,
 ['10.138.0.73', '10.138.0.71'],
 2)

In [25]:
os.remove('temp.txt')

In [27]:
all_ip_list = new_server_only_ips + new_client_only_ips

In [30]:
with open("ifconfig.txt", "w") as file:
    for item in all_ip_list:
        file.write(item + "\n")

In [5]:
    
    
    os.system('git add .; git commit -m "fixes"; git push -f origin HEAD:master')

    def setup_remote(i):

        command = 'gcloud compute ssh --zone "'+str(node_info_dict['region'][i].strip()[1:-1])+'" "resdb-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "sudo apt -y update;sudo apt -y install build-essential; sudo add-apt-repository ppa:ubuntu-toolchain-r/test -y; sudo apt update; sudo apt install g++-7 -y; sudo rm -r resDB; git clone https://github.com/draznm/resDB.git; cd resDB; cd deps && \ls | xargs -i tar -xvf {}"'

        print(command)

        output = os.system(command)
        print(output)
        
        
    def git_pull(i):

        command = 'gcloud compute ssh --zone "'+str(node_info_dict['region'][i].strip()[1:-1])+'" "resdb-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "cd resDB; git pull"'

        print(command)

        output = os.system(command)
        print(output)
            
            
    results = Parallel(n_jobs=60)(delayed(setup_remote)(i)\
                                  for i in node_info_dict['node_id'])

{'node_id': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9}, 'region': {0: " 'us-west1-b'", 1: " 'us-west1-b'", 2: " 'us-west1-b'", 3: " 'us-west1-b'", 4: " 'us-west1-b'", 5: " 'us-west1-b'", 6: " 'us-west1-b'", 7: " 'us-west1-b'", 8: " 'us-west1-b'", 9: " 'us-west1-b'"}, 'ServerOrClientregion': {0: 0, 1: 0, 2: 0, 3: 0, 4: 1, 5: 0, 6: 0, 7: 0, 8: 0, 9: 1}, 'cluster_id': {0: 0, 1: 0, 2: 0, 3: 0, 4: -1, 5: 1, 6: 1, 7: 1, 8: 1, 9: -1}, 'key_file': {0: " 'FKA.pem'", 1: " 'FKA.pem'", 2: " 'FKA.pem'", 3: " 'FKA.pem'", 4: " 'FKA.pem'", 5: "'FKA.pem'", 6: "'FKA.pem'", 7: "'FKA.pem'", 8: "'FKA.pem'", 9: " 'FKA.pem'"}, 'clientID': {0: -1, 1: -1, 2: -1, 3: -1, 4: 0, 5: -1, 6: -1, 7: -1, 8: -1, 9: 1}, 'ServerID': {0: 0, 1: 1, 2: 2, 3: 3, 4: -1, 5: 4, 6: 5, 7: 6, 8: 7, 9: -1}, ' PublicIp': {0: nan, 1: nan, 2: nan, 3: nan, 4: nan, 5: nan, 6: nan, 7: nan, 8: nan, 9: nan}}


Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/instances/resdb-002].
 - Disk size: '20 GB' is larger than image size: '10 GB'. You might need to resize the root repartition manually if the operating system does not support automatic resizing. See https://cloud.google.com/compute/docs/disks/add-persistent-disk#resize_pd for details.



NAME       ZONE        MACHINE_TYPE   PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP    STATUS
resdb-002  us-west1-b  e2-standard-2               10.138.0.3   34.145.27.251  RUNNING


Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/instances/resdb-001].
Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/instances/resdb-008].
Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/instances/resdb-005].
Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/instances/resdb-003].
Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/instances/resdb-004].
Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/instances/resdb-006].
Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/instances/resdb-007].
 - Disk size: '20 GB' is larger than image size: '10 GB'. You might need to resize the root repartition manually if the operating system does not support

NAME       ZONE        MACHINE_TYPE   PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP  STATUS
resdb-001  us-west1-b  e2-standard-2               10.138.0.63  34.127.15.0  RUNNING
NAME       ZONE        MACHINE_TYPE   PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP    STATUS
resdb-008  us-west1-b  e2-standard-2               10.138.0.7   35.203.157.14  RUNNING
NAME       ZONE        MACHINE_TYPE   PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP    STATUS
resdb-005  us-west1-b  e2-standard-2               10.138.0.60  34.83.188.221  RUNNING
NAME       ZONE        MACHINE_TYPE   PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP   STATUS
resdb-003  us-west1-b  e2-standard-2               10.138.0.59  34.83.13.221  RUNNING


...

Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/instances/resdb-000].
 - Disk size: '20 GB' is larger than image size: '10 GB'. You might need to resize the root repartition manually if the operating system does not support automatic resizing. See https://cloud.google.com/compute/docs/disks/add-persistent-disk#resize_pd for details.



NAME       ZONE        MACHINE_TYPE   PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP    STATUS
resdb-000  us-west1-b  e2-standard-2               10.138.0.61  34.83.104.142  RUNNING


Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-west1-b/instances/resdb-009].
 - Disk size: '20 GB' is larger than image size: '10 GB'. You might need to resize the root repartition manually if the operating system does not support automatic resizing. See https://cloud.google.com/compute/docs/disks/add-persistent-disk#resize_pd for details.



NAME       ZONE        MACHINE_TYPE   PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP    STATUS
resdb-009  us-west1-b  e2-standard-2               10.138.0.56  34.168.37.201  RUNNING


NameError: name 'redirect' is not defined

In [14]:
    def git_pull(i):

        command = 'gcloud compute ssh --zone "'+str(node_info_dict['region'][i].strip()[1:-1])+'" "resdb-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "cd resDB; git pull"'

        print(command)

        output = os.system(command)
        print(output)
            
            
    results = Parallel(n_jobs=60)(delayed(git_pull)(i)\
                                  for i in node_info_dict['node_id'])

From https://github.com/draznm/resDB
   1904d20..4da5c9e  master     -> origin/master
From https://github.com/draznm/resDB
   1904d20..4da5c9e  master     -> origin/master
From https://github.com/draznm/resDB
   1904d20..4da5c9e  master     -> origin/master
From https://github.com/draznm/resDB
   1904d20..4da5c9e  master     -> origin/master
From https://github.com/draznm/resDB
   1904d20..4da5c9e  master     -> origin/master
From https://github.com/draznm/resDB
   1904d20..4da5c9e  master     -> origin/master
From https://github.com/draznm/resDB
   1904d20..4da5c9e  master     -> origin/master
From https://github.com/draznm/resDB
   1904d20..4da5c9e  master     -> origin/master
From https://github.com/draznm/resDB
   1904d20..4da5c9e  master     -> origin/master
From https://github.com/draznm/resDB
   1904d20..4da5c9e  master     -> origin/master


Updating 1904d20..4da5c9e
Fast-forward
Updating 1904d20..4da5c9e
Fast-forward
Updating 1904d20..4da5c9e
Fast-forward
Updating 1904d20..4da5c9e
Fast-forward
Updating 1904d20..4da5c9e
Fast-forward
Updating 1904d20..4da5c9e
Fast-forward
Updating 1904d20..4da5c9e
Fast-forward
Updating 1904d20..4da5c9e
Fast-forward
Updating 1904d20..4da5c9e
Fast-forward
Updating 1904d20..4da5c9e
Fast-forward
 .gitignore      |      1 -
 ifconfig.txt    |     10 +
 resdb_job.ipynb | 310604 +----------------------------------------------------
 3 files changed, 117 insertions(+), 310498 deletions(-)
 create mode 100644 ifconfig.txt
 .gitignore      |      1 -
 ifconfig.txt    |     10 +
 resdb_job.ipynb | 310604 +----------------------------------------------------
 3 files changed, 117 insertions(+), 310498 deletions(-)
 create mode 100644 ifconfig.txt
 .gitignore      |      1 -
 ifconfig.txt    |     10 +
 resdb_job.ipynb | 310604 +----------------------------------------------------
 3 files changed, 117 